# A people assignment problem.
## Copyright (C) Princeton Consultants, 2017-2018

- Assign hours of work of people to sites per day of week
- Each person can work
  - At most a specified number of hours per day
  - At most a specified number of hours per week
  - At most a specified number of days per week
- Each site needs a set of people assigned for a number of hours per day
- When a person is assigned to a site
  - They must be assigned for at least an hour, in half-hour increments
  - They must be assigned for at least 2 days in the week
- There is a preference score for each person/site pair that is eligible to be assigned
  - Maximize the preferences
- There are not enough people available to cover all the site requirements, so maximize the coverage as best as possible 


## The list of files:
* sites.csv   contains site data:  ID and num hours per week
* people.csv  contains people data:  ID, total hours per week, days per week
* peoplehoursperday.csv contains hours per day for each person:  ID, day, hours
* sitehoursperday.csv contains hours per day for each site:  ID, day, hours
* prefs.csv contains the allowed assignments and a preference score:  person_id, site_id, pref


In [3]:
import pandas as pd
sites = pd.read_csv("sites.csv", index_col=0)
sites.index.name = 'site_id'
people = pd.read_csv("people.csv", index_col=0)
people.index.name = 'person_id'
peoplehoursperday = pd.read_csv("peoplehoursperday.csv", index_col=[0,1])
peoplehoursperday.index.names = ['person_id', 'day']
sitehoursperday = pd.read_csv("sitehoursperday.csv", index_col=[0,1])
sitehoursperday.index.names = ['site_id', 'day']
prefs = pd.read_csv("prefs.csv", index_col=[0,1])

In [4]:
sites.head()

,num_hours
site_id,
TWD0000000357BVYIM,12.0
UXE0000000358CWZJN,179.0
VYF0000000359DXAKO,171.0
WZG0000000360EYBLP,11.0
XAH0000000361FZCMQ,10.0


In [5]:
people.head()

,total_hrs_per_week,days_per_week
person_id,,
ADK0000000000ICFPT,6,3
BEL0000000001JDGQU,12,3
CFM0000000002KEHRV,10,7
DGN0000000003LFISW,28,6
EHO0000000004MGJTX,14,1


In [6]:
sitehoursperday.head()

hours
site_id            day       
ADK0000000364ICFPT Mo     8.0
                   Tu     8.0
                   We     8.0
                   Th     8.0
                   Fr     4.0

In [7]:
peoplehoursperday.head()

,,hours
person_id,day,
ADK0000000000ICFPT,Mo,0
BEL0000000001JDGQU,Mo,8
CFM0000000002KEHRV,Mo,0
DGN0000000003LFISW,Mo,0
EHO0000000004MGJTX,Mo,0


In [8]:
prefs.head()

,,pref
person_id,site_id,
MPW0000000272UORBF,ADK0000000364ICFPT,64
ORY0000000196WQTDH,ADK0000000364ICFPT,32
QTA0000000146YSVFJ,ADK0000000364ICFPT,97
TWD0000000123BVYIM,ADK0000000364ICFPT,57
ADK0000000000ICFPT,ADK0000000390ICFPT,35


## Data checks.  Do people and peoplehoursperday correspond?  Compare the indexes

In [9]:
uniquePeople = set(people.index.get_level_values('person_id'))
print ("PeopleHoursPerDay and People refer to same people? ", 
       set(peoplehoursperday.index.get_level_values('person_id')) == uniquePeople)
print ("Number of people ", len(people))

PeopleHoursPerDay and People refer to same people?  True
Number of people  357


## Check that the people in the prefs table are in the people table (and see if some people never are). 


In [10]:
peopleInPrefs = set(prefs.index.get_level_values('person_id'))
print ("People in Prefs and People refer to same people? ", 
       (len(uniquePeople) == len(peopleInPrefs)) and
       (uniquePeople == peopleInPrefs))
print ("Number of people in prefs = ", len(peopleInPrefs))

People in Prefs and People refer to same people?  False
Number of people in prefs =  338


## This means that some people can never be assigned.  Now check that the people in the prefs table are in the people table

In [11]:
print ("People in prefs are in people table? ",
       peopleInPrefs.issubset(uniquePeople))


People in prefs are in people table?  True


## Time to check the sites.  Do same checks as with people, but in reverse

In [12]:
uniqueSites = set(sites.index.get_level_values('site_id'))
uniqueSitesPerDay = set(sitehoursperday.index.get_level_values('site_id'))
print ("SiteHoursPerDay and Sites refer to same sites? ",
       (len(uniqueSites) == len(uniqueSitesPerDay)) and
       uniqueSitesPerDay == uniqueSites)
print ("Number of sites ", len(uniqueSites))
print ("Number of sites from Hours Per day ", len(uniqueSitesPerDay))

SiteHoursPerDay and Sites refer to same sites?  False
Number of sites  548
Number of sites from Hours Per day  498


## So there are some sites that have no openings on any days.  Let's make a list of them

In [13]:
sitesWithNoDays = uniqueSites.difference(uniqueSitesPerDay)
print("Number of sites with no daily hours = ", len(sitesWithNoDays))

Number of sites with no daily hours =  50


## So we have seen that there are 50 sites that have no days they can be assigned.  Now check the sites against the prefs table

In [14]:
sitesInPrefs = set(prefs.index.get_level_values('site_id'))
print("Number of sites from prefs = ", len(sitesInPrefs))

Number of sites from prefs =  498


## Take a guess that the data is consistent in this regard

# The next step is to take the prefs table (eligible pairs), merge with the peoplehoursperday and the sitehoursperday, and determine how many hours can possibly be assigned to each pair (the min of the two values)

In [21]:
prefswithdaily = (
    pd.merge(prefs.reset_index(), peoplehoursperday.reset_index(), on='person_id', how='left')
    .rename(columns={'hours' : 'person_hours'})
    .merge(sitehoursperday.reset_index(), on=['site_id','day'], how='inner')
    .rename(columns={'hours' : 'site_hours'})
    )
prefswithdaily['daily_hours'] = prefswithdaily[['person_hours','site_hours']].min(axis=1)
print("Length of prefs with daily, including zeros = ", len(prefswithdaily))
prefswithdaily = prefswithdaily.drop(['person_hours','site_hours'],axis=1)[prefswithdaily.daily_hours > 0]
print("Length of prefs with daily, without zeros = ",len(prefswithdaily))
prefswithdaily.head(20)

Length of prefs with daily, including zeros =  112364
Length of prefs with daily, without zeros =  87621


,person_id,site_id,pref,day,daily_hours
0,MPW0000000272UORBF,ADK0000000364ICFPT,64,Mo,8.0
2,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,Mo,6.0
3,TWD0000000123BVYIM,ADK0000000364ICFPT,57,Mo,6.0
4,MPW0000000272UORBF,ADK0000000364ICFPT,64,Tu,8.0
6,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,Tu,6.0
7,TWD0000000123BVYIM,ADK0000000364ICFPT,57,Tu,6.0
8,MPW0000000272UORBF,ADK0000000364ICFPT,64,We,8.0
9,ORY0000000196WQTDH,ADK0000000364ICFPT,32,We,4.0
10,QTA0000000146YSVFJ,ADK0000000364ICFPT,97,We,6.0
11,TWD0000000123BVYIM,ADK0000000364ICFPT,57,We,6.0


In [23]:
import sys
sys.path.append("C:/EclipseWorkspaces/LiClipseWorkspace/OptiPandas/src")
import optipandas as opd
opd.init('GUROBI')


In [24]:
from gurobipy import *
gurobipy.gurobi.version()

(7, 5, 2)

In [25]:
allvars = prefswithdaily.set_index(['person_id','site_id','day'])
allvars.head()

,,,pref,daily_hours
person_id,site_id,day,,
MPW0000000272UORBF,ADK0000000364ICFPT,Mo,64,8.0
QTA0000000146YSVFJ,ADK0000000364ICFPT,Mo,97,6.0
TWD0000000123BVYIM,ADK0000000364ICFPT,Mo,57,6.0
MPW0000000272UORBF,ADK0000000364ICFPT,Tu,64,8.0
QTA0000000146YSVFJ,ADK0000000364ICFPT,Tu,97,6.0


In [26]:
eligiblePairs = set((p,s) for (p,s,d) in allvars.index)
eligiblePeople = set(p for p,s in eligiblePairs)
eligibleSites = set(s for p,s in eligiblePairs)
print (len(eligiblePairs), len(eligiblePeople), len(eligibleSites))

31427 337 498


In [28]:
m=Model("sched")

### Declare h and pairday
    dvar int+ h[<p,s,d> in allVarsIndex] in 0..blocksPerPersonSiteDay[<p,s,d>];
    dvar int+ pairday[<p,s,d> in allVarsIndex] in 0..1;


In [19]:
h = pd.Series(m.addVars(allvars.index, 
                         ub=2*allvars.daily_hours.values,
                         vtype=GRB.INTEGER,
                         name="h").values(),
              index=allvars.index, name='h')
pairday = pd.Series(m.addVars(allvars.index,
                               vtype=GRB.BINARY,
                               name='pairday').values(),
                    index=allvars.index, name='pairday')
m.update()

In [20]:
pd.set_option('display.max_colwidth', 1000)
h.head(10)

person_id           site_id             day
MPW0000000272UORBF  ADK0000000364ICFPT  Mo     <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Mo]>
                                        Tu     <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Tu]>
                                        We     <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,We]>
                                        Th     <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Th]>
                                        Fr     <gurobi.Var h[MPW0000000272UORBF,ADK0000000364ICFPT,Fr]>
ORY0000000196WQTDH  ADK0000000364ICFPT  We     <gurobi.Var h[ORY0000000196WQTDH,ADK0000000364ICFPT,We]>
                                        Th     <gurobi.Var h[ORY0000000196WQTDH,ADK0000000364ICFPT,Th]>
                                        Fr     <gurobi.Var h[ORY0000000196WQTDH,ADK0000000364ICFPT,Fr]>
QTA0000000146YSVFJ  ADK0000000364ICFPT  Mo     <gurobi.Var h[QTA0000000146YSVFJ,ADK0000000364ICFPT,Mo]>
                    

In [21]:
pairday.head()

person_id           site_id             day
MPW0000000272UORBF  ADK0000000364ICFPT  Mo     <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Mo]>
                                        Tu     <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Tu]>
                                        We     <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,We]>
                                        Th     <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Th]>
                                        Fr     <gurobi.Var pairday[MPW0000000272UORBF,ADK0000000364ICFPT,Fr]>
Name: pairday, dtype: object

### Declare z
    dvar int+ z[eligiblePairs] in 0..1;


In [22]:
z = pd.Series(m.addVars(eligiblePairs, vtype=GRB.BINARY, name="z").values(), 
              index=pd.MultiIndex.from_tuples(eligiblePairs, names=['person_id','site_id']),
              name='z')
m.update()
z.head()

person_id           site_id           
NQX0000000065VPSCG  MPW0000000402UORBF    <gurobi.Var z[NQX0000000065VPSCG,MPW0000000402UORBF]>
KNU0000000088SMPZD  RUB0000000823ZTWGK    <gurobi.Var z[KNU0000000088SMPZD,RUB0000000823ZTWGK]>
VYF0000000281DXAKO  SVC0000000824AUXHL    <gurobi.Var z[VYF0000000281DXAKO,SVC0000000824AUXHL]>
EHO0000000056MGJTX  EHO0000000706MGJTX    <gurobi.Var z[EHO0000000056MGJTX,EHO0000000706MGJTX]>
RUB0000000069ZTWGK  HKR0000000397PJMWA    <gurobi.Var z[RUB0000000069ZTWGK,HKR0000000397PJMWA]>
Name: z, dtype: object

### Declare shortage and excess
    dvar float+ shortage[eligibleSites];
    dvar float+ excess[eligibleSites];


In [23]:
shortage = pd.Series(m.addVars(eligibleSites, vtype=GRB.CONTINUOUS, name='shortage').values(),
                     index=pd.Index(eligibleSites, name='site_id'),
                     name='shortage')
excess = pd.Series(m.addVars(eligibleSites, vtype=GRB.CONTINUOUS, name='excess').values(),
                  index=pd.Index(eligibleSites, name='site_id'),
                  name='excess')

m.update()
print("Number of variables ", m.NumVars)
pd.DataFrame({'shortage' : shortage, 'excess' : excess}).head()


Number of variables  207665


,excess,shortage
site_id,,
RUB0000000797ZTWGK,<gurobi.Var excess[RUB0000000797ZTWGK]>,<gurobi.Var shortage[RUB0000000797ZTWGK]>
GJQ0000000396OILVZ,<gurobi.Var excess[GJQ0000000396OILVZ]>,<gurobi.Var shortage[GJQ0000000396OILVZ]>
XAH0000000439FZCMQ,<gurobi.Var excess[XAH0000000439FZCMQ]>,<gurobi.Var shortage[XAH0000000439FZCMQ]>
HKR0000000657PJMWA,<gurobi.Var excess[HKR0000000657PJMWA]>,<gurobi.Var shortage[HKR0000000657PJMWA]>
EHO0000000654MGJTX,<gurobi.Var excess[EHO0000000654MGJTX]>,<gurobi.Var shortage[EHO0000000654MGJTX]>


## Define the objective function

### Initial objective
    minimize 1.0*totalShortage + 0.0*totalExcess + 0.0*totalPrefs;
    subject to {
    	totalShortage == sum(s in eligibleSites) shortage[s];


In [24]:
m.setObjective(quicksum(shortage), sense=GRB.MINIMIZE)


### PersonHoursPerDay
	forall (<p,d> in prefPeopleHoursByDayIndex) {
	PersonHoursPerDay[<p,d>]:		
		sum(<p,s,d> in allVarsIndex) h[<p,s,d>] <= 2*hours_per_people_day[<p,d>];
	}


In [25]:
opd.forall(m, ["person_id", "day"], (opd.sum('site_id', h) <= 2*peoplehoursperday.hours), "PersonHoursPerDay")
m.update()
print(m.NumConstrs)


1271


### SiteHoursPerDay
	forall (<s,d> in prefSitesHoursByDayIndex) {
	SiteHoursPerDay[<s,d>]:	
		sum(<p,s,d> in allVarsIndex) h[<p,s,d>] <= 2*hours_per_site_day[<s,d>];
	}


In [26]:
opd.forall(m, ["site_id", "day"], (opd.sum('person_id', h) <= 2*sitehoursperday.hours), "SiteHoursPerDay")
m.update()
print(m.NumConstrs)


2975


### TotalHoursPerWeek
	forall (p in eligiblePeople) {
	TotalHoursPerWeek[p]:	
		sum(<p,s,d> in allVarsIndex) h[<p,s,d>] <= 2*peopleData[p].total_hrs_per_week;
	}


In [27]:
opd.forall(m, "person_id", (opd.sum(['site_id','day'], h) <= 2*people.total_hrs_per_week), "TotalHoursPerWeek")
m.update()
print(m.NumConstrs)


3312


### PairDay Constraints
	forall (<p,s,d> in allVarsIndex) {
	PairDayUp[<p,s,d>]:	
		h[<p,s,d>] <= blocksPerPersonSiteDay[<p,s,d>]*pairday[<p,s,d>];
	PairDayLow[<p,s,d>]:
		2*pairday[<p,s,d>] <= h[<p,s,d>];
	}


In [28]:
opd.forall(m, ["person_id", "site_id", "day"], (h <= 2*allvars.daily_hours*pairday), "PairDayUp")
opd.forall(m, ["person_id", "site_id", "day"], (2*pairday <= h), "PairDayLow")
m.update()
print(m.NumConstrs)

178554


### Total Days Per Week
	forall (p in eligiblePeople) {
	TotalDaysPerWeek[p]:
		sum(<p,s,d> in allVarsIndex) pairday[<p,s,d>] <= peopleData[p].days_per_week;
	}			


In [29]:
opd.forall(m, "person_id" ,(opd.sum(["site_id", "day"], pairday) <= people.days_per_week), "TotalDaysPerWeek")
m.update()
print(m.NumConstrs)

178891


### ZVarUpper and Lower
#### Compute the sum expression once for efficiency
	forall (<p,s> in eligiblePairs) {
	ZVarUp[<p,s>]:	
		2*z[<p,s>] <= sum(<p,s,d> in allVarsIndex) pairday[<p,s,d>];
	ZVarLow[<p,s>]:
		sum(<p,s,d> in allVarsIndex) pairday[<p,s,d>] <= card({<p,s,d> | <p,s,d> in allVarsIndex})*z[<p,s>];
	}
#### Compute the sum expression once for efficiency


In [30]:
sumpairday = opd.sum("day", pairday)
opd.forall(m, ["person_id", "site_id"], (2*z <= sumpairday), 'ZVarUp')
sumpairdayBigM = sumpairday.apply(lambda v : v.size())
opd.forall(m, ["person_id", "site_id"], (sumpairday <= sumpairdayBigM*z), 'ZVarLow')
m.update()
print(m.NumConstrs)


241745


### Define the shortage constraints, for each site, measure how well we can cover

### SiteHours
	forall (s in eligibleSites) {
	SiteHours[s]:	
		sum(<p,s,d> in allVarsIndex) h[<p,s,d>] + shortage[s] - excess[s] == 2*num_hours_per_site[s];	
	}		

In [31]:
opd.forall(m, ["site_id"], (opd.sum(["person_id", "day"], h) + shortage - excess == 2*sites.num_hours), "SiteHours")
m.update()
print(m.NumConstrs)


242243


In [32]:

m.setParam(GRB.Param.TimeLimit, 300)
m.setParam(GRB.Param.CutPasses, 1)


Changed value of parameter TimeLimit to 300.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter CutPasses to 1
   Prev: -1  Min: -1  Max: 2000000000  Default: -1


In [33]:
m.write("complex.gurobi.new.lp")

In [34]:
m.optimize()

Optimize a model with 242243 rows, 207665 columns and 1027681 nonzeros
Variable types: 996 continuous, 206669 integer (119048 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 4e+02]
Found heuristic solution: objective 9934.0000000
Presolve removed 115647 rows and 87952 columns
Presolve time: 3.99s
Presolved: 126596 rows, 119713 columns, 619845 nonzeros
Variable types: 0 continuous, 119713 integer (69513 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8506    4.2000008e+03   1.974215e+03   0.000000e+00      5s
   18817    4.2500000e+03   0.000000e+00   0.000000e+00      8s

Root relaxation: objective 4.250000e+03, 18817 iterations, 2.97 seconds
Total elapsed time = 18.86s
Total elapsed time = 33.29s
Total elapsed time = 42.02s
Total elapsed time = 54.79s

    Nodes    |    Current Node    |     Objective Bounds      |     Work


### Add constraint on shortage, and set new objective to minimize excess
	shortageBound:
		totalShortage <= initshortageBound ;
	cplex.setObjCoef(thisOplModel.totalShortage, 0);
	cplex.setObjCoef(thisOplModel.totalExcess, 1);
	thisOplModel.shortageBound.UB = objVal + 0.25;


In [35]:
m.addConstr(quicksum(shortage) <= m.ObjVal+0.25, name='shortageBound')
m.setObjective(quicksum(excess))


In [36]:
resetParams()
m.setParam(GRB.Param.TimeLimit, 180)
m.setParam(GRB.Param.Presolve, 0)
m.setParam(GRB.Param.Cuts, 0)

m.optimize()

Reset all parameters to their default values
Changed value of parameter TimeLimit to 180.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Optimize a model with 242244 rows, 207665 columns and 1028179 nonzeros
Variable types: 996 continuous, 206669 integer (119048 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 4e+03]

Loaded MIP start with objective 385

Variable types: 996 continuous, 206669 integer (123834 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.915938e+02   0.000000e+00     23s
    3334    0.0000000e+00   5.441563e+03   0.000000e+00     26s
   18937    1.4527739e-03   8.675231e+02   0.000000e+00     30s
   26816    0.0000

### Add constraint on excess amount and set new objective
	excessBound:
        totalExcess <= initshortageBound;
    thisOplModel.excessBound.UB = objVal + 0.25;
	cplex.setObjCoef(thisOplModel.totalExcess, 0);
	cplex.setObjCoef(thisOplModel.totalPrefs, -1); // To maximize

    totalPrefs == sum(p in eligiblePairs) pref[p]*z[p];
#### Note that pref[] is not indexed on eligiblePairs, so we have to pull only those pairs in the product

In [37]:
m.addConstr(quicksum(excess) <= m.ObjVal+0.25, name='excessBound')
m.setObjective(quicksum(prefs.pref[z.index]*z), sense=GRB.MAXIMIZE)


In [38]:
resetParams()
m.setParam(GRB.Param.MIPGap, 0.01) # Set gap to 1%
m.setParam(GRB.Param.Cuts, 0)
m.setParam(GRB.Param.SubMIPNodes, 1500)
m.setParam(GRB.Param.TimeLimit, 180)
m.optimize()

Reset all parameters to their default values
Changed value of parameter MIPGap to 0.01
   Prev: 0.0001  Min: 0.0  Max: 1e+100  Default: 0.0001
Changed value of parameter Cuts to 0
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter SubMIPNodes to 1500
   Prev: 500  Min: 0  Max: 2000000000  Default: 500
Changed value of parameter TimeLimit to 180.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 242245 rows, 207665 columns and 1028677 nonzeros
Variable types: 996 continuous, 206669 integer (119048 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 4e+03]

Loaded MIP start with objective 22428

Presolve removed 98729 rows and 79141 columns
Presolve time: 3.12s
Presolved: 143516 rows, 128524 columns, 656494 nonzeros
Variable types: 738 continuous, 127786 integer (69472 binary)

Root simplex log...

Iteration    Objective       Primal 

In [39]:
hvals = pd.Series([hv.x for hv in h], index=h.index)
pairings = hvals[hvals>0]
pd.DataFrame(pairings.head(10),columns=['hours'])

hours
person_id          site_id            day       
MPW0000000272UORBF ADK0000000364ICFPT Mo    16.0
                                      We     6.0
ORY0000000196WQTDH ADK0000000364ICFPT We     2.0
                                      Fr     4.0
TWD0000000123BVYIM ADK0000000364ICFPT Tu    12.0
                                      Th    12.0
BEL0000000235JDGQU ADK0000000390ICFPT Tu    16.0
                                      We    12.0
CFM0000000210KEHRV ADK0000000390ICFPT Mo    14.0
                                      We     2.0

In [40]:
del m
disposeDefaultEnv()


Freed default Gurobi environment
